In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, roc_curve, recall_score, auc, roc_auc_score

!ls ../results

1.jpg
berk.csv
berk-test.csv
GoogleLenetv2.png
GoogLeNet_binary61_feature_imagenet_all.npy
GoogLeNet_binary61_feature_imagenet_test7600.npy
googlenet_weights.npy
logs
metrics
models
myRAP_labels_pd.csv
OEDCWPAL_PETA_features_all.pickle
PA-100K_labels_pd.csv
partGoogleLenet.png
PETA_coarse_to_fine_labels_pd.csv
PETA.csv
PETA_labels_pd.csv
PETA_lowerBody_labels_pd.csv
PETA_ratio_positive_samples_for_attributes.json
predictions
RAP_labels_pd.csv
RAP_partion.npy
RAP_partion.txt
README.md
relation_array.npy
samples
state_transition_matrix.npy
test_predictions.csv
title.csv


In [2]:
import scipy.io as sio

data = sio.loadmat('/home/anhaoran/data/pedestrian_attributes_PA-100K/annotation/annotation.mat')
attributes_list = []
for i in range(data['attributes'].shape[0]):
    attributes_list.append(data['attributes'][i][0][0])
print(attributes_list)

['Female', 'AgeOver60', 'Age18-60', 'AgeLess18', 'Front', 'Side', 'Back', 'Hat', 'Glasses', 'HandBag', 'ShoulderBag', 'Backpack', 'HoldObjectsInFront', 'ShortSleeve', 'LongSleeve', 'UpperStride', 'UpperLogo', 'UpperPlaid', 'UpperSplice', 'LowerStripe', 'LowerPattern', 'LongCoat', 'Trousers', 'Shorts', 'Skirt&Dress', 'boots']


In [3]:
low_level = [15,16,17,18,19,20]
mid_level = [7,8,9,10,11,12,13,14,21,22,23,24,25]
high_level = [0,1,2,3,4,5,6]
attributes_list_hiar = list(np.array(attributes_list)[list(np.hstack((low_level, mid_level, high_level)))])
print("The total attributes labeled: ", len(attributes_list_hiar))
print(attributes_list_hiar)

The total attributes labeled:  26
['UpperStride', 'UpperLogo', 'UpperPlaid', 'UpperSplice', 'LowerStripe', 'LowerPattern', 'Hat', 'Glasses', 'HandBag', 'ShoulderBag', 'Backpack', 'HoldObjectsInFront', 'ShortSleeve', 'LongSleeve', 'LongCoat', 'Trousers', 'Shorts', 'Skirt&Dress', 'boots', 'Female', 'AgeOver60', 'Age18-60', 'AgeLess18', 'Front', 'Side', 'Back']


In [4]:
filename = r"../results/PA-100K_labels_pd.csv"
data = np.array(pd.read_csv(filename))[:, 1:]
length = len(data)
data_y = np.zeros((length, 26))
for i in range(length):
    data_y[i] = np.array(data[i, 1:27], dtype="float32")
data_y_hiar = data_y[:, list(np.hstack((low_level, mid_level, high_level)))]
y_test = data_y[90000:]
y_test_hiar = data_y_hiar[90000:]
print("The shape of the y_test is: ", y_test.shape)
print("The shape of the y_test is: ", y_test_hiar.shape)

The shape of the y_test is:  (10000, 26)
The shape of the y_test is:  (10000, 26)


# Defining the function

In [5]:
def mA(y_pred, y_true):
    M = len(y_pred)
    L = len(y_pred[0])
    res = 0
    for i in range(L):
        P = sum(y_true[:, i])
        N = M - P
        TP = sum(y_pred[:, i]*y_true[:, i])
        TN = list(y_pred[:, i]+y_true[:, i] == 0).count(True)
        #print(P,',', N,',', TP,',', TN)
        if P != 0:
            res += TP/P + TN/N
        else:
            res += TN/N
    return res / (2*L)

def acc(y_pred, y_true):
    M = len(y_pred)
    M_ = 0
    res = 0
    for i in range(M):
        #print(np.shape(y_pred[i]*y_true[i]))
        if sum(y_pred[i])+sum(y_true[i])-sum(y_pred[i]*y_true[i]) != 0:
            res += sum(y_pred[i]*y_true[i]) / (sum(y_pred[i])+sum(y_true[i])-sum(y_pred[i]*y_true[i]))
            M_ += 1
    return res / M_

def prec(y_pred, y_true):
    M = len(y_pred)
    M_ = 0
    res = 0
    for i in range(M):
        if sum(y_pred[i]) != 0:
            res += sum(y_pred[i]*y_true[i]) / sum(y_pred[i])
            M_ += 1
    if M_ == 0:
        return 0
    return res / M_

def rec(y_pred, y_true):
    M = len(y_pred)
    M_ = 0
    res = 0
    for i in range(M):
        if sum(y_true[i]) != 0:
            res += sum(y_pred[i]*y_true[i]) / sum(y_true[i])
            M_ += 1
    if M_ == 0:
        return 0
    return res / M_

# Metrics

In [12]:
###hiarBayesGoogLeNet
predictions_prob = np.load("../results/predictions/hiarBayesGoogLeNet_binary26binary26_final100iter_model_predictions_imagenet_test_PA-100K.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")
label = y_test_hiar
print("hiarBayesGoogLeNet")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

###hiarGoogLeNet
predictions_prob = np.load("../results/predictions/hiarGoogLeNet_binary26binary26_epoch25_valloss0.63_predictions_imagenet_test_PA-100K.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")
label = y_test_hiar
print("hiarGoogLeNet")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))


###GoogLeNet
predictions_prob = np.load("../results/predictions/GoogLeNet_binary26binary26_final100iter_model_predictions_imagenet_test_PA-100K.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")
label = y_test
print("GoogLeNet")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

hiarBayesGoogLeNet
0.6662840253577108
0.46376119442347835
0.5367915535853012
0.7362076190476379
0.6208802646313543
hiarGoogLeNet
0.7118341933921332
0.4805365118507233
0.5407130383668619
0.7737682142857333
0.6365804933221295
GoogLeNet
0.7041406329591706
0.5441982226961998
0.6360220954045973
0.7505980158730396
0.6885763721142463


In [11]:
data_dict = {"attributes": attributes_list_hiar}

###hiarBayesGoogLeNet
predictions_prob = np.load("../results/predictions/hiarBayesGoogLeNet_binary26_binary26_final50iter_model_predictions_imagenet_test_PA-100K.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected]
label = y_test_hiar#[:, selected]

print(predictions)
print(label)

metric = []
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    metric.append(accuracy_score(predictions[:, i], label[:, i]))
metric = np.asarray(metric)
data_dict.update({"hiarBayesGoogLeNet": metric})

###GoogLeNet
predictions_prob = np.load("../results/predictions/GoogLeNet_binary26_binary26_trainable__final50iter_model_predictions_imagenet_test_PA-100K.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected_]
label = y_test#[:, selected_]

print(predictions)
print(label)

metric = []
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    metric.append(accuracy_score(predictions[:, i], label[:, i]))
metric = np.asarray(metric)[list(np.hstack((low_level, mid_level, high_level)))]
data_dict.update({"GoogLeNet": metric})

df = pd.DataFrame(data_dict)
df.set_index('attributes', inplace=True)
df

[[0. 0. 0. ... 0. 1. 0.]
 [0. 1. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
[[0. 0. 0. ... 0. 1. 0.]
 [0. 1. 0. ... 1. 0. 0.]
 [0. 1. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [1. 0. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]]
[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 ...
 [1. 0. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]]


,hiarBayesGoogLeNet,GoogLeNet
attributes,,
UpperStride,0.9500,0.9281
UpperLogo,0.8640,0.8767
UpperPlaid,0.8491,0.8768
UpperSplice,0.9610,0.9688
LowerStripe,0.9949,0.9976
LowerPattern,0.9878,0.9861
Hat,0.9686,0.9638
Glasses,0.7892,0.7687
HandBag,0.8253,0.8384


In [12]:
print(np.mean(data_dict['GoogLeNet']))
print(np.mean(data_dict['hiarBayesGoogLeNet']))

0.899376923076923
0.896126923076923
